# Step 3: Train model with Ensemble Features

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import mxnet as mx
from mxnet import gluon
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet.gluon import nn

import h5py
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [6]:
ctx = mx.gpu() # GPU WORKING HERE

![image](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/argmax.PNG)
![image](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/entropy_softmax.PNG)

In [7]:
 # calculate the acc of model
def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar() 
    # assclar() -> transofrm arrary to coresspoding scalar
    # argmax() -> find the maxmum value in muti dimentional arrary

# calculate the loss and acc of model
def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter) # iteration
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar() # Calculate cross entropy of softmax output and one-hot label.
    return loss/steps, acc/steps # return average loss and acc

# Load features vector in step 2

In [8]:
with h5py.File('features_train.h5', 'r') as f:
    #features_vgg = np.array(f['vgg'])
    features_resnet = np.array(f['resnet'])
    features_densenet = np.array(f['densenet'])
    features_inception = np.array(f['inception'])
    labels = np.array(f['labels']) # label of different dogs

In [9]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" # display muti result in jupyter

**Shape of features vector**

**Vgg is a old model, therefore using other 3 models to train new model here**

In [22]:
#features_vgg.shape
labels.shape
features_resnet.shape
features_densenet.shape
features_inception.shape

(10222L,)

(10222L, 2048L)

(10222L, 2208L)

(10222L, 2048L)

**Image-> Features of densenet, resnet and inception. And then Merge to obtain a new feature vector**

In [11]:
# Extract first two cloums as features
features_resnet = features_resnet.reshape(features_resnet.shape[:2])
features_inception = features_inception.reshape(features_inception.shape[:2])
# Merge 3 features
features = np.concatenate([features_resnet, features_densenet, features_inception], axis=-1)

In [16]:
features_densenet.shape
features_resnet.shape
features_inception.shape
features.shape

(10222L, 2208L)

(10222L, 2048L)

(10222L, 2048L)

(10222L, 6304L)

## Constrct iterator by ArrayDataset and DataLoader

In [17]:
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2) # use 20% data for validation data

#constrct dataset
dataset_train = gluon.data.ArrayDataset(nd.array(X_train), nd.array(y_train))
dataset_val = gluon.data.ArrayDataset(nd.array(X_val), nd.array(y_val))

batch_size = 128 # Size of mini-batch
data_iter_train = gluon.data.DataLoader(dataset_train, batch_size, shuffle=True) # shuffle sort of dataset
data_iter_val = gluon.data.DataLoader(dataset_val, batch_size)

## Constrct Model
![image](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/softmax_loss.PNG)

**And we need set optimize and learning rate in [gluon.Trainer](https://zh.gluon.ai/chapter_optimization/adam-gluon.html)**

In [33]:
net = nn.Sequential() # define a Sequential network
with net.name_scope():
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dropout(0.5)) # prevent overfitting
    net.add(nn.Dense(120)) # 120 breeds dogs classification

net.initialize(ctx=ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 1e-4, 'wd': 1e-5})
net

Sequential(
  (0): Dense(None -> 256, Activation(relu))
  (1): Dropout(p = 0.5)
  (2): Dense(None -> 120, linear)
)

## Train Model
![image](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/record.PNG)

In [34]:
for epoch in range(50): # iterat 50 times
    # initialize parameters
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train)
    for data, label in data_iter_train: # shuffled train data with batch size 128
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        
        with autograd.record(): # auto derivation of loss
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        
        loss.backward() # derivation
        trainer.step(batch_size) 
        # 因为loss.backward()求出的梯度是一个batch的梯度之和，所以需要除以batch_size得到平均梯度，就在trainer.step(batch_size)中这样表示
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)
    
    val_loss, val_acc = evaluate(net, data_iter_val)
    print("Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%%" % (
        epoch+1, train_loss/steps, train_acc/steps*100, val_loss, val_acc*100))

Epoch 1. loss: 4.5870, acc: 5.29%, val_loss 3.8906, val_acc 28.16%
Epoch 2. loss: 3.4818, acc: 24.70%, val_loss 2.5015, val_acc 63.03%
Epoch 3. loss: 2.3550, acc: 47.39%, val_loss 1.4636, val_acc 78.58%
Epoch 4. loss: 1.6194, acc: 62.59%, val_loss 0.9511, val_acc 84.06%
Epoch 5. loss: 1.2121, acc: 70.86%, val_loss 0.7209, val_acc 85.97%
Epoch 6. loss: 0.9733, acc: 75.26%, val_loss 0.5839, val_acc 87.73%
Epoch 7. loss: 0.8166, acc: 78.99%, val_loss 0.5180, val_acc 87.82%
Epoch 8. loss: 0.7202, acc: 80.89%, val_loss 0.4620, val_acc 89.68%
Epoch 9. loss: 0.6612, acc: 81.90%, val_loss 0.4350, val_acc 89.24%
Epoch 10. loss: 0.5952, acc: 83.82%, val_loss 0.4122, val_acc 89.59%
Epoch 11. loss: 0.5422, acc: 84.93%, val_loss 0.3886, val_acc 89.49%
Epoch 12. loss: 0.5066, acc: 85.46%, val_loss 0.3788, val_acc 89.34%
Epoch 13. loss: 0.4878, acc: 86.22%, val_loss 0.3684, val_acc 89.54%
Epoch 14. loss: 0.4365, acc: 87.81%, val_loss 0.3545, val_acc 89.68%
Epoch 15. loss: 0.4178, acc: 87.97%, val_los

# Load feature vector of test data

In [35]:
with h5py.File('features_test.h5', 'r') as f:
    #features_vgg_test = np.array(f['vgg'])
    features_resnet_test = np.array(f['resnet'])
    features_densenet_test = np.array(f['densenet'])
    features_inception_test = np.array(f['inception'])

**Features mergy same as training part**

In [36]:
features_resnet_test = features_resnet_test.reshape(features_resnet_test.shape[:2])
features_inception_test = features_inception_test.reshape(features_inception_test.shape[:2])

features_test = np.concatenate([features_resnet_test, features_densenet_test, features_inception_test], axis=-1)

## Prediction

In [37]:
%%time
output = nd.softmax(net(nd.array(features_test).as_in_context(ctx))).asnumpy()

Wall time: 1.02 s


In [39]:
output.shape

(10357L, 120L)

**integration to submission format**

In [44]:
%%time
df = pd.read_csv('sample_submission.csv')

for i, c in enumerate(df.columns[1:]):
    df[c] = output[:,i]
    
#df.to_csv('pred.csv', index=None)

Wall time: 733 ms


In [45]:
df

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,3.338809e-08,1.363769e-07,4.968185e-08,1.148647e-08,2.780667e-10,5.390003e-08,9.177727e-08,6.770148e-08,1.868445e-08,...,4.450668e-08,2.936551e-08,8.700171e-09,2.749717e-07,2.467090e-09,8.945009e-08,8.161407e-09,9.549304e-08,2.376628e-08,1.251092e-07
1,00102ee9d8eb90812350685311fe5890,7.239021e-08,1.030238e-08,6.921762e-09,5.885484e-08,2.247034e-08,5.021575e-08,8.854153e-08,8.726333e-09,2.504320e-08,...,1.415466e-07,1.383130e-08,4.042690e-08,6.139040e-07,6.584788e-08,9.285507e-08,6.131715e-06,1.298342e-07,7.373106e-09,2.473148e-08
2,0012a730dfa437f5f3613fb75efcd4ce,1.761396e-09,2.193402e-04,4.916691e-07,2.203355e-08,2.959070e-09,1.149469e-08,2.526606e-08,7.704136e-09,2.198103e-07,...,1.965128e-08,5.901514e-07,8.150218e-08,1.232072e-06,8.416451e-08,4.186788e-05,1.164154e-08,8.455832e-07,6.475479e-08,3.942433e-08
3,001510bc8570bbeee98c8d80c8a95ec1,2.408999e-04,3.334786e-07,2.043313e-07,1.719671e-07,4.063948e-06,9.020300e-07,8.348402e-07,1.580270e-06,1.995657e-06,...,2.180770e-06,7.387077e-08,1.329163e-05,3.773214e-06,4.217096e-08,1.720226e-07,6.978020e-07,8.456096e-06,4.466418e-08,1.407552e-07
4,001a5f3114548acdefa3d4da05474c2e,5.872858e-02,1.650531e-05,4.963626e-07,4.443505e-07,5.373401e-07,7.837461e-06,1.826465e-04,4.362540e-05,2.521674e-06,...,3.347674e-05,1.232059e-05,2.311019e-06,1.160790e-05,2.713161e-07,4.173835e-06,8.993936e-05,8.305112e-05,1.446322e-05,4.892287e-04
5,00225dcd3e4d2410dd53239f95c0352f,5.303541e-06,4.330623e-03,3.825677e-06,7.040541e-04,3.575608e-05,1.184968e-05,2.657193e-05,8.425179e-05,4.762358e-05,...,3.521474e-02,9.517459e-06,6.694744e-06,3.752566e-05,7.517276e-06,4.661342e-06,6.168486e-06,5.187510e-06,2.016747e-04,8.614146e-05
6,002c2a3117c2193b4d26400ce431eebd,3.927715e-07,1.799073e-07,2.242376e-08,3.807708e-07,4.901096e-08,3.611114e-08,9.996958e-01,8.003313e-08,1.412641e-09,...,6.167809e-10,1.829100e-07,9.745698e-08,1.082523e-07,1.830247e-09,3.516850e-08,7.074903e-08,1.177175e-08,8.094646e-07,2.870009e-05
7,002c58d413a521ae8d1a5daeb35fc803,1.560831e-06,3.612611e-07,8.677345e-08,7.700099e-07,5.193937e-07,4.404285e-07,1.776615e-06,1.675802e-07,1.238933e-06,...,2.621426e-06,1.216539e-07,2.505230e-07,3.047260e-06,1.034301e-06,5.608574e-06,3.653043e-04,2.177934e-06,1.205146e-06,9.694315e-07
8,002f80396f1e3db687c5932d7978b196,3.354319e-08,2.925018e-07,2.639096e-06,1.646592e-06,1.238219e-07,9.054934e-08,4.328432e-08,3.917257e-10,4.610802e-08,...,1.247321e-08,2.804119e-08,3.729168e-06,2.884877e-07,2.181859e-06,8.379592e-08,1.084471e-07,2.308173e-06,4.329720e-07,4.686183e-08
9,0036c6bcec6031be9e62a257b1c3c442,4.857402e-10,6.359738e-08,3.664490e-07,3.467145e-09,4.758820e-07,4.969432e-09,7.052568e-08,2.922750e-09,3.934375e-07,...,2.548908e-07,4.233171e-09,3.136413e-06,7.126030e-08,2.053677e-06,2.363686e-07,7.543873e-09,1.584674e-09,3.784119e-08,8.982182e-10


**Loss is 0.26657**

---

![image](https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/rank_1.PNG)

# Play with this model

In [46]:
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [47]:
ctx = mx.cpu() # use cpu   gpu will cause python stop working here

preprocessing = [
    image.ForceResizeAug((224,224)), # Make resize shorter edge to size augmenter.
    image.ColorNormalizeAug(mean=nd.array([0.485, 0.456, 0.406]), std=nd.array([0.229, 0.224, 0.225])) # Mean and std normalization
]

def transform(data, label):
    data = data.astype('float32') / 255  # transform type to float and do normalization
    for pre in preprocessing:
        data = pre(data) # preprocessing data
    
    data = nd.transpose(data, (2,0,1)) # Transpose image to N, H, W  Kaggle image is BGR but we need RGB here
    return data, nd.array([label]).asscalar().astype('float32') # return to ndARRARY

def get_features(net, data):
    features = []
    labels = []

    for X, y in tqdm(data): # tqdm to display the schedule
        feature = net.features(X.as_in_context(ctx)) # return target arrary of image features
        features.append(feature.asnumpy()) # asnumpy() -> convert to numpy array  superposition features
        labels.append(y.asnumpy()) # superposition labels
    
    features = np.concatenate(features, axis=0) # splice big data fast
    labels = np.concatenate(labels, axis=0)
    return features, labels

In [52]:
preprocessing[0] = image.ForceResizeAug((224,224))
imgs = vision.ImageFolderDataset('for_test', transform=transform)
data = gluon.data.DataLoader(imgs, 64)

#features_vgg, _ = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)
features_resnet_test, _ = get_features(models.resnet152_v1(pretrained=True, ctx=ctx), data)
features_densenet_test, _ = get_features(models.densenet161(pretrained=True, ctx=ctx), data)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.08s/it]


In [53]:
preprocessing[0] = image.ForceResizeAug((299,299))
imgs_299 = vision.ImageFolderDataset('for_test', transform=transform)
data_299 = gluon.data.DataLoader(imgs_299, 64)

features_inception_test, _ = get_features(models.inception_v3(pretrained=True, ctx=ctx), data)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.06s/it]


In [54]:
features_resnet_test = features_resnet_test.reshape(features_resnet_test.shape[:2])
features_inception_test = features_inception_test.reshape(features_inception_test.shape[:2])

features_test = np.concatenate([features_resnet_test, features_densenet_test, features_inception_test], axis=-1)

In [55]:
features_test.shape

(9L, 6304L)

In [59]:
%%time
ctx = mx.gpu()
output = nd.softmax(net(nd.array(features_test).as_in_context(ctx))).asnumpy()

Wall time: 7 ms


In [61]:
df = pd.read_csv('play.csv')

for i, c in enumerate(df.columns[1:]):
    df[c] = output[:,i]
    
df

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,gakki,0.003300,2.146267e-03,2.374506e-02,4.693337e-03,1.467181e-03,5.373989e-02,3.295791e-03,1.068859e-02,9.301095e-04,...,0.027639,7.054830e-04,4.325586e-03,4.445586e-02,4.072518e-04,1.535987e-03,0.001510,8.106782e-02,7.456132e-04,0.075522
1,lbj,0.010391,3.426854e-03,3.375243e-03,3.388047e-03,1.024398e-02,2.715941e-02,1.742182e-03,1.356574e-02,1.189596e-03,...,0.010381,1.482023e-02,1.788112e-03,5.032847e-04,1.256199e-02,7.862886e-03,0.004149,2.245429e-02,7.563993e-04,0.006772
2,lbj_dog,0.002522,2.641871e-03,2.419023e-02,6.603371e-04,1.211232e-05,3.938686e-03,1.117741e-03,1.864619e-03,4.635472e-04,...,0.001561,1.189174e-03,2.393864e-04,1.622491e-02,1.833896e-04,1.213803e-02,0.003025,1.089650e-03,2.595427e-04,0.022409
3,minghui,0.002394,1.505301e-03,2.959822e-03,1.297245e-03,1.244449e-03,2.990543e-02,2.967193e-02,1.066965e-02,6.665795e-04,...,0.000460,6.300951e-05,3.236760e-03,1.686336e-02,1.086515e-03,3.462723e-04,0.000417,1.717968e-02,2.195678e-02,0.016392
4,shiyuan,0.001131,9.817295e-02,1.452859e-02,2.528736e-04,1.650364e-04,2.471526e-02,6.515169e-03,2.078568e-02,8.253477e-04,...,0.048205,9.456469e-04,9.945760e-04,2.322860e-03,7.821865e-05,3.896379e-03,0.005804,1.843704e-03,7.732998e-04,0.057777
5,sjw,0.002359,2.539508e-03,3.031914e-03,2.468147e-04,1.971229e-03,2.586129e-02,1.602074e-03,2.243339e-01,7.441605e-03,...,0.007852,1.251200e-03,7.197974e-04,1.414346e-02,6.925876e-05,7.288409e-03,0.018366,8.551370e-03,5.842790e-03,0.005577
6,wifi,0.000003,1.934447e-07,2.280943e-08,2.860270e-06,3.943605e-07,4.538414e-07,2.819758e-06,3.786133e-07,1.058913e-06,...,0.007200,8.603908e-07,3.960791e-07,6.462409e-07,9.855820e-07,6.743753e-07,0.000056,1.048744e-06,2.104093e-07,0.000025
7,wifi_1,0.000001,8.924700e-08,7.369625e-09,2.967036e-07,1.479544e-08,4.251804e-08,9.052362e-07,8.970682e-08,9.339616e-09,...,0.000281,2.829024e-07,2.562145e-07,5.826296e-08,5.146887e-08,2.088315e-08,0.000002,3.687507e-07,6.574365e-08,0.000006
8,zjm,0.001544,1.453925e-02,2.367935e-04,2.430328e-04,1.788755e-03,1.716408e-03,6.008908e-03,1.152762e-03,2.270173e-03,...,0.002381,5.656866e-05,2.209296e-02,6.531134e-02,2.054053e-04,8.650993e-05,0.002278,2.512626e-01,2.411311e-03,0.019337


In [73]:
df1 = pd.read_csv('play_pred.csv')

In [106]:
wifi_1 = df[7:8]
del wifi_1['id']
wifi_1.max(axis=1)

7    0.999366
dtype: float32

99.9366% **pomeranian**  in predicted csv file

<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/wifi_1.jpg" width="400px" align="left"/>
<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/pomeranian.jpg" width="400px" align="middle"/>

In [102]:
lbj_dog = df[2:3]
del lbj_dog['id']
lbj_dog.max(axis=1)

2    0.181071
dtype: float32

0.181071 coresspoding **pembroke**  in predicted csv file

<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/lbj_dog.jpg" width="300px" align="left"/>
<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/pembroke.jpg" width="350px" align="middle"/>

In [100]:
gakki = df[0:1]
del gakki['id']
gakki.max(axis=1)

0    0.081068
dtype: float32

0.081068 coresspoding **whippet** in predicted csv file

<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/gakki.jpg" width="200px" align="left"/>
<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/whippet.jpg" width="250px" align="middle"/>

In [101]:
lbj = df[1:2]
del lbj['id']
lbj.max(axis=1)

1    0.070452
dtype: float32

0.070452 coresspoding **italian_greyhound**  in predicted csv file

<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/lbj.jpg" width="200px" align="left"/>
<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/italian_greyhound.jpg" width="300px" align="middle"/>

In [104]:
shiyuan = df[4:5]
del shiyuan['id']
shiyuan.max(axis=1)

4    0.138123
dtype: float32

0.138123 coresspoding **japanese_spaniel**  in predicted csv file

<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/shiyuan.jpg" width="400px" align="left"/>
<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/japanese_spaniel.jpg" width="150px" align="middle"/>

In [105]:
sjw = df[5:6]
del sjw['id']
sjw.max(axis=1)

5    0.224334
dtype: float32

 0.224334 coresspoding **basenji**  in predicted csv file

<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/sjw.jpg" width="400px" align="left"/>
<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/basenji.jpg" width="400px" align="middle"/>

In [107]:
zjm = df[8:9]
del zjm['id']
zjm.max(axis=1)

8    0.251263
dtype: float32

0.251263 coresspoding **whippet** in predicted csv file

<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/zjm.jpg" width="220px" align="left"/>
<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/whippet.jpg" width="250px" align="middle"/>

In [103]:
minghui = df[3:4]
del minghui['id']
minghui.max(axis=1)

3    0.255215
dtype: float32

0.255215 coresspoding **eskimo_dog**  in predicted csv file

<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/minghui.jpg" width="200px" align="left"/>
<img src="https://raw.githubusercontent.com/Trouble404/Kaggle-Dog-breed-Identification/master/readme_pic_add/eskimo_dog.jpg" width="350px" align="middle"/>